In [1]:
!pip install transformers chromadb langchain langchain_community pypdf openai tiktoken rapidocr-onnxruntime

In [2]:
# Create Vector Store
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Agent
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper



In [3]:
os.environ['OPENAI_API_KEY'] = ''

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# tokenizer = AutoTokenizer.from_pretrained("BioMistral/BioMistral-7B")
# model = AutoModel.from_pretrained("BioMistral/BioMistral-7B")
llm = OpenAI(temperature=0)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [6]:
# embeddings = SentenceTransformerEmbeddings(model_name="neuml/pubmedbert-base-embeddings-matryoshka")
embeddings = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [50]:
def create_vector_db(path,uploaded_documents=1):
  DATA_DIR = path
  documents = []

  if uploaded_documents == 1:
    for file in os.listdir(DATA_DIR):
      if file.endswith('.pdf'):
        print(os.path.join(DATA_DIR, file))
        pdf_path = os.path.join(DATA_DIR, file)
        loader = PyPDFLoader(pdf_path, extract_images=True)
        documents.extend(loader.load())
  else:
    loader = PyPDFLoader(path, extract_images=True)
    documents.extend(loader.load())

  text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
  chunked_documents = text_splitter.split_documents(documents)
  print(chunked_documents)
  client = chromadb.Client()
  print(client.list_collections())
  if client.list_collections():
    print("Collection already exists")
  else:
    print("Collection created")
    consent_collection = client.create_collection("vector_database")
  vectordb = Chroma.from_documents(
          documents=chunked_documents,
          embedding=embeddings,
          persist_directory="/content/drive/My Drive/LLM Project/VectorDataBase/chroma_store/")
  vectordb.persist()

  return vectordb

In [51]:
docsearch = create_vector_db('/content/drive/MyDrive/LLM Project/Research Papers')
#docsearch = create_vector_db('https://arxiv.org/pdf/2402.12352.pdf',uploaded_documents=0)

/content/drive/MyDrive/LLM Project/Research Papers/Impact of mechanical engineering innovations in biomedical advancements.pdf
[Document(page_content='Vol.:(0123456789)In vitro models (2024) 3:5–18 \nhttps://doi.org/10.1007/s44164-024-00065-4\nREVIEWS\nImpact of\xa0mechanical engineering innovations in\xa0biomedical \nadvancements\nSenthil\xa0Maharaj\xa0Kennedy1 \xa0· A\xa0Vasanthanathan2\xa0· RB\xa0Jeen\xa0Robert3\xa0· A\xa0Vignesh\xa0Moorthi\xa0Pandian1', metadata={'source': '/content/drive/MyDrive/LLM Project/Research Papers/Impact of mechanical engineering innovations in biomedical advancements.pdf', 'page': 0}), Document(page_content='Received: 1 December 2023 / Revised: 28 December 2023 / Accepted: 2 January 2024 / Published online: 25 January 2024 \n© The Author(s), under exclusive licence to Springer Nature Switzerland AG 2024\nAbstract', metadata={'source': '/content/drive/MyDrive/LLM Project/Research Papers/Impact of mechanical engineering innovations in biomedical advancemen

In [52]:
researchPapers = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [53]:
tools = [
    Tool(
        name = "Biomedical Research Papers",
        func=researchPapers.run,
        description="useful for when you need to answer questions about any biomedical research paper"
    )
]

In [37]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [38]:
agent.run("What are some of the mostly used prosthetic polymers?")



> Entering new AgentExecutor chain...
 I should search for research papers on prosthetic polymers
Action: Biomedical Research Papers
Action Input: "prosthetic polymers"
Observation:  Prosthetic polymers are synthetic materials that are used to create prosthetics, such as artificial limbs or joints, that can be implanted into the body. These polymers are designed to degrade over time inside the body, once their intended purpose is complete. Some commonly used prosthetic polymers include polypropylene, polyethylene, Teflon, polymethyl methacrylate, silicone, and nylon. These materials are chosen for their biocompatibility and ability to degrade safely within the body.
Thought: I should read more about the properties and uses of these polymers
Action: Biomedical Research Papers
Action Input: "properties and uses of prosthetic polymers"
Observation:  Prosthetic polymers are synthetic materials that are used to create prosthetics, such as artificial limbs or joints. They are chosen for th

'Some of the mostly used prosthetic polymers include polypropylene, polyethylene, Teflon, polymethyl methacrylate, silicone, and nylon. These polymers are chosen for their biocompatibility and ability to degrade safely within the body, and are used in a variety of prosthetic devices.'

In [13]:
agent.run("What are Types of prosthesis?")



> Entering new AgentExecutor chain...
 I should use Biomedical Research Papers to find information about types of prosthesis.
Action: Biomedical Research Papers
Action Input: Types of prosthesis
Observation:  The types of prosthesis include external and internal prosthesis, which can be further classified into artificial limbs, orthopedic implants, dental implants, nasal implants, and vascular implants.
Thought: I should read through the different types of prosthesis and understand their functions.
Action: Biomedical Research Papers
Action Input: Functions of different types of prosthesis
Observation:  External prosthesis deals with artificial limbs and is used externally, while internal prosthesis deals with internal body implants. They both aim to provide good strength and enhance aesthetic and functional fronts for the human body.
Thought: I now know the different types of prosthesis and their functions.
Final Answer: The types of prosthesis include external and internal prosthesi

'The types of prosthesis include external and internal prosthesis, which can be further classified into artificial limbs, orthopedic implants, dental implants, nasal implants, and vascular implants. They aim to provide good strength and enhance aesthetic and functional fronts for the human body.'

In [14]:
agent.run("What is the research paper about?")



> Entering new AgentExecutor chain...
 I should read the abstract and introduction to get an idea of the topic
Action: Biomedical Research Papers
Action Input: Abstract and Introduction
Observation:  The context provided is from a scientific journal article discussing the challenges of keeping up with the latest discoveries in biomedical research due to the overwhelming amount of literature available. The article also mentions the importance of identifying new connections between different biomedical entities.
Thought: I should also skim through the results and conclusion sections to get a better understanding of the main findings and implications of the research.
Action: Biomedical Research Papers
Action Input: Results and Conclusion
Observation:  The results and conclusion of this experiment are that Hybrid IR outperforms both ES IR and KG IR for smaller volumes of retrieved information. This supports the hypothesis that Hybrid IR is more effective in retrieving information from hi

'The research paper is about the challenges of keeping up with the latest discoveries in biomedical research and the effectiveness of Hybrid IR in retrieving information from high-density regions.'

In [12]:
agent.run("Can you summarize the paper in about 100 words")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should read the abstract and conclusion to get a general idea of the paper
Action: Biomedical Research Papers
Action Input: Abstract and Conclusion
Observation:  I don't know.
Thought: I should read the introduction and methods to get a better understanding of the paper
Action: Biomedical Research Papers
Action Input: Introduction and Methods
Observation:  The introduction and methods section describes the typical workflow for information retrieval (IR) and the rationale for using a knowledge graph in the IR process. The workflow consists of two steps: retrieval and synthesis, and the knowledge graph is used to address limitations in traditional text embedding similarity approaches. The methods section also mentions the use of a large corpus of text, specifically Pubmed, for the IR process.
Thought: I should read the results and discussion to see what the paper found and what it means
Action: Biomedical Research Papers
Action Input: Results an

'The paper discusses the use of a knowledge graph in the information retrieval process and how it addresses limitations in traditional text embedding similarity approaches. It also mentions the use of a large corpus, specifically Pubmed, and the imbalance of information in different topics within the corpus.'

In [13]:
# Chatbot

In [61]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain
from transformers import  AutoModelForCausalLM, AutoTokenizer

In [55]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [56]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [57]:
agent_chain.run(input="What are some of the mostly used prosthetic polymers?")



> Entering new AgentExecutor chain...
Thought: I should use the Biomedical Research Papers tool to find information on prosthetic polymers.
Action: Biomedical Research Papers
Action Input: "prosthetic polymers"
Observation:  a) Hip Prosthesis, b) Knee Prosthesis, d) Dental Implant, f) Artificial Stent
Thought: I should narrow down my search to focus on the most commonly used prosthetic polymers.
Action: Biomedical Research Papers
Action Input: "prosthetic polymers" AND "most commonly used"
Observation:  Polypropylene, polyethylene, Teflon, polymethyl methacrylate, silicone, and nylon are the most commonly used prosthetic polymers.
Thought: I now know the final answer.
Final Answer: The most commonly used prosthetic polymers are polypropylene, polyethylene, Teflon, polymethyl methacrylate, silicone, and nylon.

> Finished chain.


'The most commonly used prosthetic polymers are polypropylene, polyethylene, Teflon, polymethyl methacrylate, silicone, and nylon.'

In [58]:
agent_chain.run(input="How many polymers did you report in the previous question?")



> Entering new AgentExecutor chain...
Thought: I should check the list of commonly used prosthetic polymers.
Action: Biomedical Research Papers
Action Input: "commonly used prosthetic polymers"
Observation:  Polypropylene, polyethylene, Teflon, polymethyl methacrylate, silicone, and nylon are the commonly used prosthetic polymers.
Thought: I now know the final answer.
Final Answer: There are six commonly used prosthetic polymers.

> Finished chain.


'There are six commonly used prosthetic polymers.'

In [59]:
agent_chain.run(input="What is the conclusion of this paper?")



> Entering new AgentExecutor chain...
Thought: I should use the Biomedical Research Papers tool to find the conclusion.
Action: Biomedical Research Papers
Action Input: "Conclusion of paper"
Observation:  The conclusion of this paper is that there is a need for further research and implementation of sustainable practices and materials in the field of prosthetic implants to reduce their environmental impact.
Thought: I now know the final answer.
Final Answer: The conclusion of this paper is that there is a need for further research and implementation of sustainable practices and materials in the field of prosthetic implants to reduce their environmental impact.

> Finished chain.


'The conclusion of this paper is that there is a need for further research and implementation of sustainable practices and materials in the field of prosthetic implants to reduce their environmental impact.'